In [ ]:
# Sources:
#     spreads: https://www.kaggle.com/tobycrabtree/nfl-scores-and-betting-data?select=spreadspoke_scores.csv
#     boxscores: https://sportsreference.readthedocs.io/en/v0.1.0/_modules/sportsreference/nfl/boxscore.html
#     sklearn: https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6
#     sklearn: https://www.ritchieng.com/pandas-scikit-learn/

In [2]:
from sportsreference.nfl.boxscore import Boxscores
from sportsreference.nfl.boxscore import Boxscore
import pandas as pd
import numpy as np

In [3]:
total_games = pd.DataFrame()
for year in range(2015, 2021) :
    if year == 2020:
        for week in range(1, 4) :
            temp_week = Boxscores(year = year, week = week)
            temp_dict = temp_week.games[str(week) + '-' + str(year)]
            temp_df = pd.DataFrame.from_dict(temp_dict)
            temp_df['year'] = year
            temp_df['week'] = week
            total_games = total_games.append(temp_df, ignore_index = True)
    else:
        for week in range(1, 18) :
            temp_week = Boxscores(year = year, week = week)
            temp_dict = temp_week.games[str(week) + '-' + str(year)]
            temp_df = pd.DataFrame.from_dict(temp_dict)
            temp_df['year'] = year
            temp_df['week'] = week
            total_games = total_games.append(temp_df, ignore_index = True)
total_games

,away_abbr,away_name,away_score,boxscore,home_abbr,home_name,home_score,losing_abbr,losing_name,winning_abbr,winning_name,year,week
0,pit,Pittsburgh Steelers,21,201509100nwe,nwe,New England Patriots,28,pit,Pittsburgh Steelers,nwe,New England Patriots,2015,1
1,gnb,Green Bay Packers,31,201509130chi,chi,Chicago Bears,23,chi,Chicago Bears,gnb,Green Bay Packers,2015,1
2,sea,Seattle Seahawks,31,201509130ram,ram,St. Louis Rams,34,sea,Seattle Seahawks,ram,St. Louis Rams,2015,1
3,mia,Miami Dolphins,17,201509130was,was,Washington Redskins,10,was,Washington Redskins,mia,Miami Dolphins,2015,1
4,kan,Kansas City Chiefs,27,201509130htx,htx,Houston Texans,20,htx,Houston Texans,kan,Kansas City Chiefs,2015,1
5,clt,Indianapolis Colts,14,201509130buf,buf,Buffalo Bills,27,clt,Indianapolis Colts,buf,Buffalo Bills,2015,1
6,car,Carolina Panthers,20,201509130jax,jax,Jacksonville Jaguars,9,jax,Jacksonville Jaguars,car,Carolina Panthers,2015,1
7,cle,Cleveland Browns,10,201509130nyj,nyj,New York Jets,31,cle,Cleveland Browns,nyj,New York Jets,2015,1
8,det,Detroit Lions,28,201509130sdg,sdg,San Diego Chargers,33,det,Detroit Lions,sdg,San Diego Chargers,2015,1
9,nor,New Orleans Saints,19,201509130crd,crd,Arizona Cardinals,31,nor,New Orleans Saints,crd,Arizona Cardinals,2015,1


In [4]:
#Finds the number of wins the given team had up to this point in the season
def numSeasonWins(year, week, team):
    small_df = total_games[(total_games['year'] == year) & 
                           (total_games['winning_name'] == team) & 
                           (total_games['week'] < week)]
    return small_df.shape[0]

In [249]:
def numSeasonLosses(year, week, team):
    small_df = total_games[(total_games['year'] == year) & 
                           (total_games['losing_name'] == team) & 
                           (total_games['week'] < week)]
    return small_df.shape[0]

In [5]:
numSeasonWins(2018, 5, 'Chicago Bears')

3

In [250]:
numSeasonLosses(2018, 5, 'Chicago Bears')

1

In [251]:
vSeasonWins = np.vectorize(numSeasonWins)
vSeasonLosses = np.vectorize(numSeasonLosses)

total_games['home_wins'] = vSeasonWins(total_games['year'].values, 
                                       total_games['week'].values, 
                                       total_games['home_name'].values)
total_games['away_wins'] = vSeasonWins(total_games['year'].values, 
                                       total_games['week'].values, 
                                       total_games['away_name'].values)
total_games['home_losses'] = vSeasonLosses(total_games['year'].values, 
                                       total_games['week'].values, 
                                       total_games['home_name'].values)
total_games['away_losses'] = vSeasonLosses(total_games['year'].values, 
                                       total_games['week'].values, 
                                       total_games['away_name'].values)
total_games

/Users/sethfriman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/sethfriman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/sethfriman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

,year,week,boxscore,away_name,away_score,home_name,home_score,losing_name,winning_name,home_wins,away_wins,home_streak_before_game,away_streak_before_game,division_matchup,home_last_season_wins,away_last_season_wins,home_win,home_losses,away_losses
0,2015,1,201509100nwe,Pittsburgh Steelers,21,New England Patriots,28,Pittsburgh Steelers,New England Patriots,0,0,0,0,0,0,0,1,0,0
1,2015,1,201509130chi,Green Bay Packers,31,Chicago Bears,23,Chicago Bears,Green Bay Packers,0,0,0,0,1,0,0,0,0,0
2,2015,1,201509130ram,Seattle Seahawks,31,St. Louis Rams,34,Seattle Seahawks,St. Louis Rams,0,0,0,0,1,0,0,1,0,0
3,2015,1,201509130was,Miami Dolphins,17,Washington Redskins,10,Washington Redskins,Miami Dolphins,0,0,0,0,0,0,0,0,0,0
4,2015,1,201509130htx,Kansas City Chiefs,27,Houston Texans,20,Houston Texans,Kansas City Chiefs,0,0,0,0,0,0,0,0,0,0
5,2015,1,201509130buf,Indianapolis Colts,14,Buffalo Bills,27,Indianapolis Colts,Buffalo Bills,0,0,0,0,0,0,0,1,0,0
6,2015,1,201509130jax,Carolina Panthers,20,Jacksonville Jaguars,9,Jacksonville Jaguars,Carolina Panthers,0,0,0,0,0,0,0,0,0,0
7,2015,1,201509130nyj,Cleveland Browns,10,New York Jets,31,Cleveland Browns,New York Jets,0,0,0,0,0,0,0,1,0,0
8,2015,1,201509130sdg,Detroit Lions,28,San Diego Chargers,33,Detroit Lions,San Diego Chargers,0,0,0,0,0,0,0,1,0,0
9,2015,1,201509130crd,New Orleans Saints,19,Arizona Cardinals,31,New Orleans Saints,Arizona Cardinals,0,0,0,0,0,0,0,1,0,0


In [252]:
total_games = total_games[['year', 'week', 'boxscore', 'away_name', 'away_score', 'home_name', 'home_score', 
                           'losing_name', 'winning_name', 'home_wins', 'away_wins', 'home_losses', 'away_losses']]

In [253]:
# finds the teams winning streak before this game started. Week 1 starts every team at 0
def findStreakBefore(year, week, team) :
    if week == 1 :
        return 0
    else  :
        game_before = total_games[(total_games['year'] == year) &
                                  (total_games['week'] < week) &
                                  ((total_games['home_name'] == team) | 
                                   (total_games['away_name'] == team))].tail(1)
        if game_before.shape[0] == 0:
            return 0
        if game_before['home_name'].iloc[0] == team:
            location = 'home'
        else :
            location = 'away'
        if (game_before['winning_name'].iloc[0] == team) & (game_before[str(location) + 
                                                                     '_streak_before_game'].iloc[0] >= 0):
            return game_before[str(location) + '_streak_before_game'].iloc[0] + 1
        elif (game_before['winning_name'].iloc[0] == team) & (game_before[str(location) + 
                                                                       '_streak_before_game'].iloc[0] < 0):
            return 1
        elif (game_before['losing_name'].iloc[0] == team) & (game_before[str(location) + 
                                                                      '_streak_before_game'].iloc[0] >= 0):
            return -1
        elif (game_before['losing_name'].iloc[0] == team) & (game_before[str(location) + 
                                                                      '_streak_before_game'].iloc[0] < 0):
            return game_before[str(location) + '_streak_before_game'].iloc[0] - 1
        else :
            return game_before[str(location) + '_streak_before_game'].iloc[0]

In [254]:
total_games[(total_games['year'] == 2018) &
                                  (total_games['week'] < 5) &
                                  ((total_games['home_name'] == 'Chicago Bears') | 
                                   (total_games['away_name'] == 'Chicago Bears'))].tail(1).shape[0]

1

In [255]:
# Adds winning streak to each team
total_games['home_streak_before_game'] = 0
total_games['away_streak_before_game'] = 0
for i in range(0, total_games.shape[0]):
    temp_game = total_games.iloc[i]
    total_games.at[i, 'home_streak_before_game'] = findStreakBefore(temp_game.year, 
                                                                      temp_game.week, 
                                                                      temp_game.home_name)
    total_games.at[i, 'away_streak_before_game'] = findStreakBefore(temp_game.year, 
                                                                      temp_game.week, 
                                                                      temp_game.away_name)
total_games

/Users/sethfriman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/sethfriman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,year,week,boxscore,away_name,away_score,home_name,home_score,losing_name,winning_name,home_wins,away_wins,home_losses,away_losses,home_streak_before_game,away_streak_before_game
0,2015,1,201509100nwe,Pittsburgh Steelers,21,New England Patriots,28,Pittsburgh Steelers,New England Patriots,0,0,0,0,0,0
1,2015,1,201509130chi,Green Bay Packers,31,Chicago Bears,23,Chicago Bears,Green Bay Packers,0,0,0,0,0,0
2,2015,1,201509130ram,Seattle Seahawks,31,St. Louis Rams,34,Seattle Seahawks,St. Louis Rams,0,0,0,0,0,0
3,2015,1,201509130was,Miami Dolphins,17,Washington Redskins,10,Washington Redskins,Miami Dolphins,0,0,0,0,0,0
4,2015,1,201509130htx,Kansas City Chiefs,27,Houston Texans,20,Houston Texans,Kansas City Chiefs,0,0,0,0,0,0
5,2015,1,201509130buf,Indianapolis Colts,14,Buffalo Bills,27,Indianapolis Colts,Buffalo Bills,0,0,0,0,0,0
6,2015,1,201509130jax,Carolina Panthers,20,Jacksonville Jaguars,9,Jacksonville Jaguars,Carolina Panthers,0,0,0,0,0,0
7,2015,1,201509130nyj,Cleveland Browns,10,New York Jets,31,Cleveland Browns,New York Jets,0,0,0,0,0,0
8,2015,1,201509130sdg,Detroit Lions,28,San Diego Chargers,33,Detroit Lions,San Diego Chargers,0,0,0,0,0,0
9,2015,1,201509130crd,New Orleans Saints,19,Arizona Cardinals,31,New Orleans Saints,Arizona Cardinals,0,0,0,0,0,0


In [256]:
total_games.home_streak_before_game.value_counts()

-1     304
 1     292
-2     153
 2     150
 0      98
-3      76
 3      63
-4      42
 4      29
-5      21
 5      21
 6      12
-6      12
 7       9
 8       9
-7       8
 9       8
-8       6
-9       4
-11      3
-12      2
-10      2
-14      1
 11      1
-13      1
 12      1
Name: home_streak_before_game, dtype: int64

In [257]:
total_games[(total_games['week'] != 1) & (total_games['away_streak_before_game'] == 0)]

,year,week,boxscore,away_name,away_score,home_name,home_score,losing_name,winning_name,home_wins,away_wins,home_losses,away_losses,home_streak_before_game,away_streak_before_game
536,2017,2,201709170sdg,Miami Dolphins,19,Los Angeles Chargers,17,Los Angeles Chargers,Miami Dolphins,0,0,1,0,-1,0
792,2018,2,201809160nor,Cleveland Browns,18,New Orleans Saints,21,Cleveland Browns,New Orleans Saints,0,0,1,0,-1,0
1043,2019,2,201909150rav,Arizona Cardinals,17,Baltimore Ravens,23,Arizona Cardinals,Baltimore Ravens,1,0,0,0,1,0


In [258]:
total_games[total_games['winning_name'] == total_games['home_name']].shape[0] / total_games.shape[0]

0.5572289156626506

In [259]:
afc_north = ['Cleveland Browns', 'Cincinnati Bengals', 'Pittsburgh Steelers', 'Baltimore Ravens']
afc_south = ['Indianapolis Colts', 'Jacksonville Jaguars', 'Houston Texans', 'Tennessee Titans']
afc_east = ['New England Patriots', 'New York Jets', 'Miami Dolphins', 'Buffalo Bills']
afc_west = ['Denver Broncos', 'Kansas City Chiefs', 'Oakland Raiders', 'Las Vegas Raiders', 'San Diego Chargers',
           'Los Angeles Chargers']
nfc_north = ['Chicago Bears', 'Minnesota Vikings', 'Detroit Lions', 'Green Bay Packers']
nfc_south = ['New Orleans Saints', 'Atlanta Falcons', 'Carolina Panthers', 'Tampa Bay Buccaneers']
nfc_east = ['New York Giants', 'Philadelphia Eagles', 'Dallas Cowboys', 'Washington Redskins', 
            'Washington Football Team']
nfc_west = ['Seattle Seahawks', 'San Francisco 49ers', 'St. Louis Rams', 'Los Angeles Rams', 'Arizona Cardinals']
divisions = {'AFC North' : afc_north, 'AFC South' : afc_south, 'AFC East' : afc_east, 'AFC West' : afc_west, 
             'NFC North' : nfc_north, 'NFC South' : nfc_south, 'NFC East' : nfc_east, 'NFC West' : nfc_west}

In [260]:
def addDivision(team) :
    for div, teams in divisions.items():
        if team in teams:
            return div

In [261]:
vDivisions = np.vectorize(addDivision)
total_games['home_division'] = vDivisions(total_games.home_name.values)
total_games['away_division'] = vDivisions(total_games.away_name.values)
total_games

/Users/sethfriman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/sethfriman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,year,week,boxscore,away_name,away_score,home_name,home_score,losing_name,winning_name,home_wins,away_wins,home_losses,away_losses,home_streak_before_game,away_streak_before_game,home_division,away_division
0,2015,1,201509100nwe,Pittsburgh Steelers,21,New England Patriots,28,Pittsburgh Steelers,New England Patriots,0,0,0,0,0,0,AFC East,AFC North
1,2015,1,201509130chi,Green Bay Packers,31,Chicago Bears,23,Chicago Bears,Green Bay Packers,0,0,0,0,0,0,NFC North,NFC North
2,2015,1,201509130ram,Seattle Seahawks,31,St. Louis Rams,34,Seattle Seahawks,St. Louis Rams,0,0,0,0,0,0,NFC West,NFC West
3,2015,1,201509130was,Miami Dolphins,17,Washington Redskins,10,Washington Redskins,Miami Dolphins,0,0,0,0,0,0,NFC East,AFC East
4,2015,1,201509130htx,Kansas City Chiefs,27,Houston Texans,20,Houston Texans,Kansas City Chiefs,0,0,0,0,0,0,AFC South,AFC West
5,2015,1,201509130buf,Indianapolis Colts,14,Buffalo Bills,27,Indianapolis Colts,Buffalo Bills,0,0,0,0,0,0,AFC East,AFC South
6,2015,1,201509130jax,Carolina Panthers,20,Jacksonville Jaguars,9,Jacksonville Jaguars,Carolina Panthers,0,0,0,0,0,0,AFC South,NFC South
7,2015,1,201509130nyj,Cleveland Browns,10,New York Jets,31,Cleveland Browns,New York Jets,0,0,0,0,0,0,AFC East,AFC North
8,2015,1,201509130sdg,Detroit Lions,28,San Diego Chargers,33,Detroit Lions,San Diego Chargers,0,0,0,0,0,0,AFC West,NFC North
9,2015,1,201509130crd,New Orleans Saints,19,Arizona Cardinals,31,New Orleans Saints,Arizona Cardinals,0,0,0,0,0,0,NFC West,NFC South


In [262]:
def lastSeasonWins(year, team):
    if team == 'Las Vegas Raiders' :
        last_season = total_games[(total_games['year'] == year - 1) & 
                                  ((total_games['winning_name'] == 'Oakland Raiders') |
                                   (total_games['winning_name'] == team))]
    elif team == 'Los Angeles Chargers' :
        last_season = total_games[(total_games['year'] == year - 1) & 
                                  ((total_games['winning_name'] == team) |
                                   (total_games['winning_name'] == 'San Diego Chargers'))]
    elif team == 'Los Angeles Rams' :
        last_season = total_games[(total_games['year'] == year - 1) & 
                                  ((total_games['winning_name'] == team) |
                                   (total_games['winning_name'] == 'St. Louis Rams'))]
    elif team == 'Washington Football Team' :
        last_season = total_games[(total_games['year'] == year - 1) & 
                                  ((total_games['winning_name'] == team) |
                                   (total_games['winning_name'] == 'Washington Redskins'))]
    else :
        last_season = total_games[(total_games['year'] == year - 1) & (total_games['winning_name'] == team)]
    return last_season.shape[0]

In [263]:
total_games['division_matchup'] = total_games['home_division'] == total_games['away_division']
total_games['division_matchup'] = total_games['division_matchup'].astype(int)

/Users/sethfriman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/sethfriman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [264]:
vLastSeasonWins = np.vectorize(lastSeasonWins)
total_games['home_last_season_wins'] = vLastSeasonWins(total_games.year.values, total_games.home_name.values)
total_games['away_last_season_wins'] = vLastSeasonWins(total_games.year.values, total_games.away_name.values)
total_games

,year,week,boxscore,away_name,away_score,home_name,home_score,losing_name,winning_name,home_wins,away_wins,home_losses,away_losses,home_streak_before_game,away_streak_before_game,home_division,away_division,division_matchup,home_last_season_wins,away_last_season_wins
0,2015,1,201509100nwe,Pittsburgh Steelers,21,New England Patriots,28,Pittsburgh Steelers,New England Patriots,0,0,0,0,0,0,AFC East,AFC North,0,0,0
1,2015,1,201509130chi,Green Bay Packers,31,Chicago Bears,23,Chicago Bears,Green Bay Packers,0,0,0,0,0,0,NFC North,NFC North,1,0,0
2,2015,1,201509130ram,Seattle Seahawks,31,St. Louis Rams,34,Seattle Seahawks,St. Louis Rams,0,0,0,0,0,0,NFC West,NFC West,1,0,0
3,2015,1,201509130was,Miami Dolphins,17,Washington Redskins,10,Washington Redskins,Miami Dolphins,0,0,0,0,0,0,NFC East,AFC East,0,0,0
4,2015,1,201509130htx,Kansas City Chiefs,27,Houston Texans,20,Houston Texans,Kansas City Chiefs,0,0,0,0,0,0,AFC South,AFC West,0,0,0
5,2015,1,201509130buf,Indianapolis Colts,14,Buffalo Bills,27,Indianapolis Colts,Buffalo Bills,0,0,0,0,0,0,AFC East,AFC South,0,0,0
6,2015,1,201509130jax,Carolina Panthers,20,Jacksonville Jaguars,9,Jacksonville Jaguars,Carolina Panthers,0,0,0,0,0,0,AFC South,NFC South,0,0,0
7,2015,1,201509130nyj,Cleveland Browns,10,New York Jets,31,Cleveland Browns,New York Jets,0,0,0,0,0,0,AFC East,AFC North,0,0,0
8,2015,1,201509130sdg,Detroit Lions,28,San Diego Chargers,33,Detroit Lions,San Diego Chargers,0,0,0,0,0,0,AFC West,NFC North,0,0,0
9,2015,1,201509130crd,New Orleans Saints,19,Arizona Cardinals,31,New Orleans Saints,Arizona Cardinals,0,0,0,0,0,0,NFC West,NFC South,0,0,0


In [265]:
total_games['home_win'] = total_games['home_score'] > total_games['away_score']
total_games['home_win'] = total_games['home_win'].astype(int)
total_games

,year,week,boxscore,away_name,away_score,home_name,home_score,losing_name,winning_name,home_wins,...,home_losses,away_losses,home_streak_before_game,away_streak_before_game,home_division,away_division,division_matchup,home_last_season_wins,away_last_season_wins,home_win
0,2015,1,201509100nwe,Pittsburgh Steelers,21,New England Patriots,28,Pittsburgh Steelers,New England Patriots,0,...,0,0,0,0,AFC East,AFC North,0,0,0,1
1,2015,1,201509130chi,Green Bay Packers,31,Chicago Bears,23,Chicago Bears,Green Bay Packers,0,...,0,0,0,0,NFC North,NFC North,1,0,0,0
2,2015,1,201509130ram,Seattle Seahawks,31,St. Louis Rams,34,Seattle Seahawks,St. Louis Rams,0,...,0,0,0,0,NFC West,NFC West,1,0,0,1
3,2015,1,201509130was,Miami Dolphins,17,Washington Redskins,10,Washington Redskins,Miami Dolphins,0,...,0,0,0,0,NFC East,AFC East,0,0,0,0
4,2015,1,201509130htx,Kansas City Chiefs,27,Houston Texans,20,Houston Texans,Kansas City Chiefs,0,...,0,0,0,0,AFC South,AFC West,0,0,0,0
5,2015,1,201509130buf,Indianapolis Colts,14,Buffalo Bills,27,Indianapolis Colts,Buffalo Bills,0,...,0,0,0,0,AFC East,AFC South,0,0,0,1
6,2015,1,201509130jax,Carolina Panthers,20,Jacksonville Jaguars,9,Jacksonville Jaguars,Carolina Panthers,0,...,0,0,0,0,AFC South,NFC South,0,0,0,0
7,2015,1,201509130nyj,Cleveland Browns,10,New York Jets,31,Cleveland Browns,New York Jets,0,...,0,0,0,0,AFC East,AFC North,0,0,0,1
8,2015,1,201509130sdg,Detroit Lions,28,San Diego Chargers,33,Detroit Lions,San Diego Chargers,0,...,0,0,0,0,AFC West,NFC North,0,0,0,1
9,2015,1,201509130crd,New Orleans Saints,19,Arizona Cardinals,31,New Orleans Saints,Arizona Cardinals,0,...,0,0,0,0,NFC West,NFC South,0,0,0,1


In [266]:
total_games.columns

Index(['year', 'week', 'boxscore', 'away_name', 'away_score', 'home_name',
       'home_score', 'losing_name', 'winning_name', 'home_wins', 'away_wins',
       'home_losses', 'away_losses', 'home_streak_before_game',
       'away_streak_before_game', 'home_division', 'away_division',
       'division_matchup', 'home_last_season_wins', 'away_last_season_wins',
       'home_win'],
      dtype='object')

In [267]:
total_games = total_games[['year', 'week', 'boxscore', 'away_name', 'away_score', 'home_name',
       'home_score', 'losing_name', 'winning_name', 'home_wins', 'home_losses', 'away_wins', 'away_losses',
       'home_streak_before_game', 'away_streak_before_game', 'division_matchup', 'home_last_season_wins',
       'away_last_season_wins', 'home_win']]

In [268]:
spread_data = pd.read_csv('spreadspoke_scores.csv')

In [269]:
spread_data = spread_data[spread_data['schedule_season'] >= 2015]
spread_data = spread_data[~spread_data['schedule_playoff']]

In [270]:
spread_data = spread_data[['schedule_season', 'schedule_week', 'team_home', 'team_away', 'team_favorite_id', 'spread_favorite', 
                           'over_under_line']]
spread_data.columns = ['year', 'week', 'home_name', 'away_name', 'team_favorite_id', 'spread_favorite', 
                           'over_under_line']
spread_data
                           

,year,week,home_name,away_name,team_favorite_id,spread_favorite,over_under_line
11343,2015,1,New England Patriots,Pittsburgh Steelers,NE,-7.0,51
11344,2015,1,Arizona Cardinals,New Orleans Saints,ARI,-2.5,48.5
11345,2015,1,Buffalo Bills,Indianapolis Colts,IND,-1.0,44.5
11346,2015,1,Chicago Bears,Green Bay Packers,GB,-6.5,48.5
11347,2015,1,Dallas Cowboys,New York Giants,DAL,-7.0,52.5
11348,2015,1,Denver Broncos,Baltimore Ravens,DEN,-4.5,46.5
11349,2015,1,Houston Texans,Kansas City Chiefs,KC,-1.0,40.5
11350,2015,1,Jacksonville Jaguars,Carolina Panthers,CAR,-3.0,41
11351,2015,1,New York Jets,Cleveland Browns,NYJ,-3.5,38.5
11352,2015,1,Oakland Raiders,Cincinnati Bengals,OAK,-3.0,43


In [271]:
abbreviations = {'ARI':' Arizona Cardinals', 
                'ATL': 'Atlanta Falcons',
                'BAL': 'Baltimore Ravens',
                'BUF': 'Buffalo Bills',
                'CAR': 'Carolina Panthers',
                'CHI': 'Chicago Bears',
                'CIN': 'Cincinnati Bengals',
                'CLE': 'Cleveland Browns',
                'DAL': 'Dallas Cowboys',
                'DEN': 'Denver Broncos',
                'DET': 'Detroit Lions',
                'GB': 'Green Bay Packers',
                'HOU': 'Houston Texans',
                'IND': 'Indianapolis Colts',
                'JAX': 'Jacksonville Jaguars',
                'KC': 'Kansas City Chiefs',
                'LAC' : 'Los Angeles Chargers',
                'LAR' : 'Los Angeles Rams',
                'LV' : 'Las Vegas Raiders',
                'MIA': 'Miami Dolphins',
                'MIN': 'Minnesota Vikings',
                'NE': 'New England Patriots',
                'NO': 'New Orleans Saints',
                'NYG': 'New York Giants',
                'NYJ':' New York Jets',
                'OAK': 'Oakland Raiders',
                'PHI': 'Philadelphia Eagles',
                'PIT': 'Pittsburgh Steelers',
                'SD': 'San Diego Chargers',
                'SEA': 'Seattle Seahawks',
                'SF': 'San Francisco 49ers',
                'STL': 'Saint Louis Rams',
                'TB': 'Tampa Bay Buccaneers',
                'TEN' : 'Tennessee Titans',
                'WAS' : 'Washington Football Team',
                'WAS': 'Washington Redskins'}

In [272]:
spread_data['favorite_name'] = spread_data.team_favorite_id.map(abbreviations)
spread_data

,year,week,home_name,away_name,team_favorite_id,spread_favorite,over_under_line,favorite_name
11343,2015,1,New England Patriots,Pittsburgh Steelers,NE,-7.0,51,New England Patriots
11344,2015,1,Arizona Cardinals,New Orleans Saints,ARI,-2.5,48.5,Arizona Cardinals
11345,2015,1,Buffalo Bills,Indianapolis Colts,IND,-1.0,44.5,Indianapolis Colts
11346,2015,1,Chicago Bears,Green Bay Packers,GB,-6.5,48.5,Green Bay Packers
11347,2015,1,Dallas Cowboys,New York Giants,DAL,-7.0,52.5,Dallas Cowboys
11348,2015,1,Denver Broncos,Baltimore Ravens,DEN,-4.5,46.5,Denver Broncos
11349,2015,1,Houston Texans,Kansas City Chiefs,KC,-1.0,40.5,Kansas City Chiefs
11350,2015,1,Jacksonville Jaguars,Carolina Panthers,CAR,-3.0,41,Carolina Panthers
11351,2015,1,New York Jets,Cleveland Browns,NYJ,-3.5,38.5,New York Jets
11352,2015,1,Oakland Raiders,Cincinnati Bengals,OAK,-3.0,43,Oakland Raiders


In [273]:
spread_data['home_favorite'] = spread_data['home_name'] == spread_data['favorite_name']
spread_data = spread_data[['year', 'week', 'home_name', 'away_name', 'favorite_name', 'spread_favorite', 
                           'over_under_line', 'home_favorite']]
spread_data

,year,week,home_name,away_name,favorite_name,spread_favorite,over_under_line,home_favorite
11343,2015,1,New England Patriots,Pittsburgh Steelers,New England Patriots,-7.0,51,True
11344,2015,1,Arizona Cardinals,New Orleans Saints,Arizona Cardinals,-2.5,48.5,False
11345,2015,1,Buffalo Bills,Indianapolis Colts,Indianapolis Colts,-1.0,44.5,False
11346,2015,1,Chicago Bears,Green Bay Packers,Green Bay Packers,-6.5,48.5,False
11347,2015,1,Dallas Cowboys,New York Giants,Dallas Cowboys,-7.0,52.5,True
11348,2015,1,Denver Broncos,Baltimore Ravens,Denver Broncos,-4.5,46.5,True
11349,2015,1,Houston Texans,Kansas City Chiefs,Kansas City Chiefs,-1.0,40.5,False
11350,2015,1,Jacksonville Jaguars,Carolina Panthers,Carolina Panthers,-3.0,41,False
11351,2015,1,New York Jets,Cleveland Browns,New York Jets,-3.5,38.5,False
11352,2015,1,Oakland Raiders,Cincinnati Bengals,Oakland Raiders,-3.0,43,True


In [274]:
spread_data['week'] = spread_data.week.astype(int)
spread_data['home_favorite'] = spread_data.home_favorite.astype(int)

/Users/sethfriman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/sethfriman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [275]:
spread_data.dtypes

year                 int64
week                 int64
home_name           object
away_name           object
favorite_name       object
spread_favorite    float64
over_under_line     object
home_favorite        int64
dtype: object

In [276]:
spread_data.week.value_counts()

17    80
16    80
15    80
14    80
2     80
3     80
1     79
13    79
12    77
4     76
7     72
6     72
5     72
10    69
11    69
8     69
9     66
Name: week, dtype: int64

In [432]:
games_and_lines = total_games.merge(spread_data, on = ['year', 'week', 'home_name', 'away_name'],
                                    how = 'outer')
games_and_lines

,year,week,boxscore,away_name,away_score,home_name,home_score,losing_name,winning_name,home_wins,...,home_streak_before_game,away_streak_before_game,division_matchup,home_last_season_wins,away_last_season_wins,home_win,favorite_name,spread_favorite,over_under_line,home_favorite
0,2015,1,201509100nwe,Pittsburgh Steelers,21,New England Patriots,28,Pittsburgh Steelers,New England Patriots,0,...,0,0,0,0,0,1,New England Patriots,-7.0,51,1.0
1,2015,1,201509130chi,Green Bay Packers,31,Chicago Bears,23,Chicago Bears,Green Bay Packers,0,...,0,0,1,0,0,0,Green Bay Packers,-6.5,48.5,0.0
2,2015,1,201509130ram,Seattle Seahawks,31,St. Louis Rams,34,Seattle Seahawks,St. Louis Rams,0,...,0,0,1,0,0,1,Seattle Seahawks,-3.5,41,0.0
3,2015,1,201509130was,Miami Dolphins,17,Washington Redskins,10,Washington Redskins,Miami Dolphins,0,...,0,0,0,0,0,0,Miami Dolphins,-4.0,45.5,0.0
4,2015,1,201509130htx,Kansas City Chiefs,27,Houston Texans,20,Houston Texans,Kansas City Chiefs,0,...,0,0,0,0,0,0,Kansas City Chiefs,-1.0,40.5,0.0
5,2015,1,201509130buf,Indianapolis Colts,14,Buffalo Bills,27,Indianapolis Colts,Buffalo Bills,0,...,0,0,0,0,0,1,Indianapolis Colts,-1.0,44.5,0.0
6,2015,1,201509130jax,Carolina Panthers,20,Jacksonville Jaguars,9,Jacksonville Jaguars,Carolina Panthers,0,...,0,0,0,0,0,0,Carolina Panthers,-3.0,41,0.0
7,2015,1,201509130nyj,Cleveland Browns,10,New York Jets,31,Cleveland Browns,New York Jets,0,...,0,0,0,0,0,1,New York Jets,-3.5,38.5,0.0
8,2015,1,201509130sdg,Detroit Lions,28,San Diego Chargers,33,Detroit Lions,San Diego Chargers,0,...,0,0,0,0,0,1,Los Angeles Chargers,-3.5,45.5,0.0
9,2015,1,201509130crd,New Orleans Saints,19,Arizona Cardinals,31,New Orleans Saints,Arizona Cardinals,0,...,0,0,0,0,0,1,Arizona Cardinals,-2.5,48.5,0.0


In [477]:
import math

In [478]:
def avgPPG(year, week, team) :
    if (week == 1) :
        if year == games_and_lines.year.min() :
            return 0
        else :
            return avgPPG(year - 1, 18, team)
    this_season_away = games_and_lines[(games_and_lines['year'] == year) &
                                         (games_and_lines['week'] < week) & 
                                         (games_and_lines['away_name'] == team)]
    this_season_home = games_and_lines[(games_and_lines['year'] == year) &
                                         (games_and_lines['week'] < week) & 
                                         (games_and_lines['home_name'] == team)]
    total_points = this_season_away.away_score.sum() + this_season_home.home_score.sum()
    avg_points = total_points / (this_season_away.shape[0] + this_season_home.shape[0])
    if (math.isnan(avg_points)) :
        return 0
    return avg_points

In [483]:
def avgPAPG(year, week, team) :
    if (week == 1) :
        if year == games_and_lines.year.min() :
            return 0
        else :
            return avgPAPG(year - 1, 18, team)
    this_season_away = games_and_lines[(games_and_lines['year'] == year) &
                                         (games_and_lines['week'] < week) & 
                                         (games_and_lines['away_name'] == team)]
    this_season_home = games_and_lines[(games_and_lines['year'] == year) &
                                         (games_and_lines['week'] < week) & 
                                         (games_and_lines['home_name'] == team)]
    total_points_allowed = this_season_away.home_score.sum() + this_season_home.away_score.sum()
    avg_points_allowed = total_points_allowed / (this_season_away.shape[0] + this_season_home.shape[0])
    if (math.isnan(avg_points_allowed)) :
        return 0
    return avg_points_allowed

In [484]:
vAvgPpg = np.vectorize(avgPPG)
vAvgPapg = np.vectorize(avgPAPG)
games_and_lines['home_avg_ppg'] = vAvgPpg(games_and_lines.year.values, 
                                          games_and_lines.week.values,
                                          games_and_lines.home_name.values)
games_and_lines['away_avg_ppg'] = vAvgPpg(games_and_lines.year.values, 
                                          games_and_lines.week.values,
                                          games_and_lines.away_name.values)
games_and_lines['home_avg_papg'] = vAvgPapg(games_and_lines.year.values, 
                                          games_and_lines.week.values,
                                          games_and_lines.home_name.values)
games_and_lines['away_avg_papg'] = vAvgPapg(games_and_lines.year.values, 
                                          games_and_lines.week.values,
                                          games_and_lines.away_name.values)
games_and_lines

/Users/sethfriman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in long_scalars
  


,year,week,boxscore,away_name,away_score,home_name,home_score,losing_name,winning_name,home_wins,...,away_last_season_wins,home_win,favorite_name,spread_favorite,over_under_line,home_favorite,home_avg_ppg,away_avg_ppg,home_avg_papg,away_avg_papg
0,2015,1,201509100nwe,Pittsburgh Steelers,21,New England Patriots,28,Pittsburgh Steelers,New England Patriots,0,...,0,1,New England Patriots,-7.0,51,1.0,0,0,0,0
1,2015,1,201509130chi,Green Bay Packers,31,Chicago Bears,23,Chicago Bears,Green Bay Packers,0,...,0,0,Green Bay Packers,-6.5,48.5,0.0,0,0,0,0
2,2015,1,201509130ram,Seattle Seahawks,31,St. Louis Rams,34,Seattle Seahawks,St. Louis Rams,0,...,0,1,Seattle Seahawks,-3.5,41,0.0,0,0,0,0
3,2015,1,201509130was,Miami Dolphins,17,Washington Redskins,10,Washington Redskins,Miami Dolphins,0,...,0,0,Miami Dolphins,-4.0,45.5,0.0,0,0,0,0
4,2015,1,201509130htx,Kansas City Chiefs,27,Houston Texans,20,Houston Texans,Kansas City Chiefs,0,...,0,0,Kansas City Chiefs,-1.0,40.5,0.0,0,0,0,0
5,2015,1,201509130buf,Indianapolis Colts,14,Buffalo Bills,27,Indianapolis Colts,Buffalo Bills,0,...,0,1,Indianapolis Colts,-1.0,44.5,0.0,0,0,0,0
6,2015,1,201509130jax,Carolina Panthers,20,Jacksonville Jaguars,9,Jacksonville Jaguars,Carolina Panthers,0,...,0,0,Carolina Panthers,-3.0,41,0.0,0,0,0,0
7,2015,1,201509130nyj,Cleveland Browns,10,New York Jets,31,Cleveland Browns,New York Jets,0,...,0,1,New York Jets,-3.5,38.5,0.0,0,0,0,0
8,2015,1,201509130sdg,Detroit Lions,28,San Diego Chargers,33,Detroit Lions,San Diego Chargers,0,...,0,1,Los Angeles Chargers,-3.5,45.5,0.0,0,0,0,0
9,2015,1,201509130crd,New Orleans Saints,19,Arizona Cardinals,31,New Orleans Saints,Arizona Cardinals,0,...,0,1,Arizona Cardinals,-2.5,48.5,0.0,0,0,0,0


In [460]:
games_and_lines.home_avg_ppg = games_and_lines.home_avg_ppg.fillna(0)
games_and_lines.away_avg_ppg = games_and_lines.away_avg_ppg.fillna(0)
games_and_lines.home_avg_papg = games_and_lines.home_avg_papg.fillna(0)
games_and_lines.away_avg_papg = games_and_lines.away_avg_papg.fillna(0)
games_and_lines

,year,week,boxscore,away_name,away_score,home_name,home_score,losing_name,winning_name,home_wins,...,away_last_season_wins,home_win,favorite_name,spread_favorite,over_under_line,home_favorite,home_avg_ppg,away_avg_ppg,home_avg_papg,away_avg_papg
0,2015,1,201509100nwe,Pittsburgh Steelers,21,New England Patriots,28,Pittsburgh Steelers,New England Patriots,0,...,0,1,New England Patriots,-7.0,51,1.0,0.0,0.0,0.0,0.0
1,2015,1,201509130chi,Green Bay Packers,31,Chicago Bears,23,Chicago Bears,Green Bay Packers,0,...,0,0,Green Bay Packers,-6.5,48.5,0.0,0.0,0.0,0.0,0.0
2,2015,1,201509130ram,Seattle Seahawks,31,St. Louis Rams,34,Seattle Seahawks,St. Louis Rams,0,...,0,1,Seattle Seahawks,-3.5,41,0.0,0.0,0.0,0.0,0.0
3,2015,1,201509130was,Miami Dolphins,17,Washington Redskins,10,Washington Redskins,Miami Dolphins,0,...,0,0,Miami Dolphins,-4.0,45.5,0.0,0.0,0.0,0.0,0.0
4,2015,1,201509130htx,Kansas City Chiefs,27,Houston Texans,20,Houston Texans,Kansas City Chiefs,0,...,0,0,Kansas City Chiefs,-1.0,40.5,0.0,0.0,0.0,0.0,0.0
5,2015,1,201509130buf,Indianapolis Colts,14,Buffalo Bills,27,Indianapolis Colts,Buffalo Bills,0,...,0,1,Indianapolis Colts,-1.0,44.5,0.0,0.0,0.0,0.0,0.0
6,2015,1,201509130jax,Carolina Panthers,20,Jacksonville Jaguars,9,Jacksonville Jaguars,Carolina Panthers,0,...,0,0,Carolina Panthers,-3.0,41,0.0,0.0,0.0,0.0,0.0
7,2015,1,201509130nyj,Cleveland Browns,10,New York Jets,31,Cleveland Browns,New York Jets,0,...,0,1,New York Jets,-3.5,38.5,0.0,0.0,0.0,0.0,0.0
8,2015,1,201509130sdg,Detroit Lions,28,San Diego Chargers,33,Detroit Lions,San Diego Chargers,0,...,0,1,Los Angeles Chargers,-3.5,45.5,0.0,0.0,0.0,0.0,0.0
9,2015,1,201509130crd,New Orleans Saints,19,Arizona Cardinals,31,New Orleans Saints,Arizona Cardinals,0,...,0,1,Arizona Cardinals,-2.5,48.5,0.0,0.0,0.0,0.0,0.0


In [278]:
games_and_lines[games_and_lines['home_favorite'] == games_and_lines['home_win']].shape[0] / games_and_lines.shape[0]

0.6265060240963856

In [162]:
season_and_box = games_and_lines[['year', 'week', 'boxscore', 'home_name', 'away_name']]
boxscores = season_and_box.boxscore.values
all_scores = pd.DataFrame()
for box in boxscores :
    temp_score = Boxscore(box).dataframe.reset_index().rename(columns = {'index' : 'boxscore'})
    temp_score = temp_score[['boxscore', 'away_first_downs', 'away_fourth_down_attempts',
       'away_fourth_down_conversions', 'away_fumbles_lost',
       'away_interceptions', 'away_pass_touchdowns', 'away_pass_yards',
       'away_penalties', 'away_rush_touchdowns', 'away_rush_yards', 'away_third_down_attempts',
       'away_third_down_conversions', 'away_time_of_possession',
       'away_times_sacked',
       'home_first_downs', 'home_fourth_down_attempts',
       'home_fourth_down_conversions', 'home_fumbles_lost',
       'home_interceptions', 'home_pass_touchdowns', 'home_pass_yards',
       'home_penalties', 'home_rush_touchdowns', 'home_rush_yards', 'home_third_down_attempts',
       'home_third_down_conversions', 'home_time_of_possession',
       'home_times_sacked']]
    all_scores = all_scores.append(temp_score, ignore_index = True)
    
all_scores

,boxscore,away_first_downs,away_fourth_down_attempts,away_fourth_down_conversions,away_fumbles_lost,away_interceptions,away_pass_touchdowns,away_pass_yards,away_penalties,away_rush_touchdowns,...,home_interceptions,home_pass_touchdowns,home_pass_yards,home_penalties,home_rush_touchdowns,home_rush_yards,home_third_down_attempts,home_third_down_conversions,home_time_of_possession,home_times_sacked
0,201509100nwe,23,2,2,0,1,1,351,8,1,...,0,4,288,7,0,80,11,7,27:55,2
1,201509130chi,21,1,1,0,0,3,189,10,1,...,1,1,225,6,1,189,17,11,31:52,2
2,201509130ram,21,2,1,0,1,1,251,7,0,...,0,1,297,4,2,76,11,6,28:32,2
3,201509130was,19,1,0,1,0,1,226,6,0,...,2,1,196,11,0,161,14,6,37:54,1
4,201509130htx,19,0,0,0,0,3,243,2,0,...,1,2,334,6,0,98,14,3,24:41,5
5,201509130buf,23,2,1,1,2,2,243,5,0,...,0,1,195,11,2,147,13,5,30:56,0
6,201509130jax,17,1,1,0,1,1,175,3,0,...,2,1,183,3,0,96,12,3,25:54,5
7,201509130nyj,15,2,1,4,1,1,231,12,0,...,1,2,179,4,2,154,13,7,28:20,0
8,201509130sdg,16,0,0,0,2,2,246,4,1,...,2,2,403,5,2,96,11,6,38:12,2
9,201509130crd,18,1,0,0,1,1,355,7,0,...,0,3,307,5,1,120,10,5,26:36,0


In [163]:
season_and_box = season_and_box.merge(all_scores, how = 'inner', on = 'boxscore')
season_and_box

,year,week,boxscore,home_name,away_name,away_first_downs,away_fourth_down_attempts,away_fourth_down_conversions,away_fumbles_lost,away_interceptions,...,home_interceptions,home_pass_touchdowns,home_pass_yards,home_penalties,home_rush_touchdowns,home_rush_yards,home_third_down_attempts,home_third_down_conversions,home_time_of_possession,home_times_sacked
0,2015,1,201509100nwe,New England Patriots,Pittsburgh Steelers,23,2,2,0,1,...,0,4,288,7,0,80,11,7,27:55,2
1,2015,1,201509130chi,Chicago Bears,Green Bay Packers,21,1,1,0,0,...,1,1,225,6,1,189,17,11,31:52,2
2,2015,1,201509130ram,St. Louis Rams,Seattle Seahawks,21,2,1,0,1,...,0,1,297,4,2,76,11,6,28:32,2
3,2015,1,201509130was,Washington Redskins,Miami Dolphins,19,1,0,1,0,...,2,1,196,11,0,161,14,6,37:54,1
4,2015,1,201509130htx,Houston Texans,Kansas City Chiefs,19,0,0,0,0,...,1,2,334,6,0,98,14,3,24:41,5
5,2015,1,201509130buf,Buffalo Bills,Indianapolis Colts,23,2,1,1,2,...,0,1,195,11,2,147,13,5,30:56,0
6,2015,1,201509130jax,Jacksonville Jaguars,Carolina Panthers,17,1,1,0,1,...,2,1,183,3,0,96,12,3,25:54,5
7,2015,1,201509130nyj,New York Jets,Cleveland Browns,15,2,1,4,1,...,1,2,179,4,2,154,13,7,28:20,0
8,2015,1,201509130sdg,San Diego Chargers,Detroit Lions,16,0,0,0,2,...,2,2,403,5,2,96,11,6,38:12,2
9,2015,1,201509130crd,Arizona Cardinals,New Orleans Saints,18,1,0,0,1,...,0,3,307,5,1,120,10,5,26:36,0


In [164]:
# converts possession time to a float
def convertPosTime(pos) :
    minutes = pos[0:pos.index(":")]
    seconds = pos[pos.index(":") + 1:]
    return int(minutes) + int(seconds)/60

In [165]:
vPosTime = np.vectorize(convertPosTime)
season_and_box['home_time_of_possession'] = vPosTime(season_and_box.home_time_of_possession)
season_and_box['away_time_of_possession'] = vPosTime(season_and_box.away_time_of_possession)
season_and_box

,year,week,boxscore,home_name,away_name,away_first_downs,away_fourth_down_attempts,away_fourth_down_conversions,away_fumbles_lost,away_interceptions,...,home_interceptions,home_pass_touchdowns,home_pass_yards,home_penalties,home_rush_touchdowns,home_rush_yards,home_third_down_attempts,home_third_down_conversions,home_time_of_possession,home_times_sacked
0,2015,1,201509100nwe,New England Patriots,Pittsburgh Steelers,23,2,2,0,1,...,0,4,288,7,0,80,11,7,27.916667,2
1,2015,1,201509130chi,Chicago Bears,Green Bay Packers,21,1,1,0,0,...,1,1,225,6,1,189,17,11,31.866667,2
2,2015,1,201509130ram,St. Louis Rams,Seattle Seahawks,21,2,1,0,1,...,0,1,297,4,2,76,11,6,28.533333,2
3,2015,1,201509130was,Washington Redskins,Miami Dolphins,19,1,0,1,0,...,2,1,196,11,0,161,14,6,37.900000,1
4,2015,1,201509130htx,Houston Texans,Kansas City Chiefs,19,0,0,0,0,...,1,2,334,6,0,98,14,3,24.683333,5
5,2015,1,201509130buf,Buffalo Bills,Indianapolis Colts,23,2,1,1,2,...,0,1,195,11,2,147,13,5,30.933333,0
6,2015,1,201509130jax,Jacksonville Jaguars,Carolina Panthers,17,1,1,0,1,...,2,1,183,3,0,96,12,3,25.900000,5
7,2015,1,201509130nyj,New York Jets,Cleveland Browns,15,2,1,4,1,...,1,2,179,4,2,154,13,7,28.333333,0
8,2015,1,201509130sdg,San Diego Chargers,Detroit Lions,16,0,0,0,2,...,2,2,403,5,2,96,11,6,38.200000,2
9,2015,1,201509130crd,Arizona Cardinals,New Orleans Saints,18,1,0,0,1,...,0,3,307,5,1,120,10,5,26.600000,0


In [190]:
atts = ['first_downs', #0,1
        'fourth_down_attempts', #2,3
        'fourth_down_conversions', #4,5
        'fumbles_lost',#6,7
        'interceptions', #8,9
        'pass_touchdowns', #10,11
        'pass_yards',#12,13
        'penalties', #14,15
        'rush_touchdowns', #16,17
        'rush_yards', #18,19
        'third_down_attempts',#20,21
        'third_down_conversions', #22,23
        'time_of_possession',#24,25
        'times_sacked']#26,27
desired_columns_away = ['year', 'week', 'boxscore', 'away_avg_first_downs', 'away_avg_fourth_down_attempts', 
       'away_avg_fourth_down_conversions', 'away_avg_fumbles_lost',
       'away_avg_interceptions_thrown', 'away_avg_pass_touchdowns', 'away_avg_pass_yards',
       'away_avg_penalties', 'away_avg_rush_touchdowns', 'away_avg_rush_yards', 'away_avg_third_down_attempts', 
       'away_avg_third_down_conversions', 'away_avg_time_of_possession', 'away_avg_times_sacked',
       'away_avg_first_downs_allowed', 'away_avg_def_fourth_down_attempts', 
       'away_avg_def_fourth_down_conversions', 'away_avg_fumbles_recovered',
       'away_avg_interceptions_taken', 'away_avg_pass_touchdowns_allowed', 'away_avg_pass_yards_allowed',
       'away_avg_rush_touchdowns_allowed', 'away_avg_rush_yards_allowed', 'away_avg_def_third_down_attempts', 
       'away_avg_def_third_down_conversions', 
       'away_avg_sacks']
desired_columns_home = ['year', 'week', 'boxscore', 'home_avg_first_downs', 'home_avg_fourth_down_attempts', 
       'home_avg_fourth_down_converstions', 'home_avg_fumbles_lost',
       'home_avg_interceptions_thrown', 'home_avg_pass_touchdowns', 'home_avg_pass_yards',
       'home_avg_penalties', 'home_avg_rush_touchdowns', 'home_avg_rush_yards', 'home_avg_third_down_attempts', 
       'home_avg_third_down_converstions', 'home_avg_time_of_possession', 'home_avg_times_sacked',
       'home_avg_first_downs_allowed', 'home_avg_def_fourth_down_attempts', 
       'home_avg_def_fourth_down_converstions', 'home_avg_fumbles_recovered',
       'home_avg_interceptions_taken', 'home_avg_pass_touchdowns_allowed', 'home_avg_pass_yards_allowed',
       'home_avg_rush_touchdowns_allowed', 'home_avg_rush_yards_allowed', 'home_avg_def_third_down_attempts', 
       'home_avg_def_third_down_converstions',
       'home_avg_sacks']

In [220]:
# find the season average for all attributes up to that point. NaN for first game ever, last season's for all
# other week 1 games.
def findAvgStats(year, week, box, team, attributes, loc) :
    att_list = []
    if (week == 1) :
        if year == games_and_lines.year.min() :
            for i in range(len(attributes) - 1):
                att_list.append(float('NaN'))
                att_list.append(float('NaN'))
            att_list = [year, week, box] + att_list
            if loc == 'away' :
                att_series = pd.Series(att_list, index = desired_columns_away)
            else :
                att_series = pd.Series(att_list, index = desired_columns_home)
            return att_series
        else :
            return findAvgStats(year - 1, 18, box, team, attributes, loc)
    else :
        this_season_away = season_and_box[(season_and_box['year'] == year) &
                                     (season_and_box['week'] < week) & 
                                     (season_and_box['away_name'] == team)]
        this_season_home = season_and_box[(season_and_box['year'] == year) &
                                     (season_and_box['week'] < week) & 
                                     (season_and_box['home_name'] == team)]
        
        for at in attributes :
            #offensive stats
            away_total_off = this_season_away[str('away_' + at)].sum()
            home_total_off = this_season_home[str('home_' + at)].sum()
            #defensive stats
            away_total_def = this_season_away[str('home_' + at)].sum()
            home_total_def = this_season_home[str('away_' + at)].sum()

            num_games = this_season_away.shape[0] + this_season_home.shape[0]
            avg_off = (away_total_off + home_total_off) / num_games
            avg_def = (away_total_def + home_total_def) / num_games
            att_list.append(avg_off)
            att_list.append(avg_def)
        new_order = [0,2,4,6,8,10,12,14,16,18,20,22,24,26,
                     1,3,5,7,9,11,13,17,19,21,23,27]
        att_list = [att_list[i] for i in new_order]
        att_list = [year, week, box] + att_list
        if loc == 'away' :
            att_series = pd.Series(att_list, index = desired_columns_away)
        else :
            att_series = pd.Series(att_list, index = desired_columns_home)
        return att_series

In [221]:
# goal: iterate thought the list only once per team per line, return all desired states in an order
# how: give all away teams first, return all desired stats. Then do the same for the home teams. Merge the two 
# dataframes at the end on year, week, and boxscore to have for both teams for all games

In [222]:
findAvgStats(2015, 1, '201509100nwe', 'Pittsburgh Steelers', atts, 'away')

year                                            2015
week                                               1
boxscore                                201509100nwe
away_avg_first_downs                             NaN
away_avg_fourth_down_attempts                    NaN
away_avg_fourth_down_conversions                 NaN
away_avg_fumbles_lost                            NaN
away_avg_interceptions_thrown                    NaN
away_avg_pass_touchdowns                         NaN
away_avg_pass_yards                              NaN
away_avg_penalties                               NaN
away_avg_rush_touchdowns                         NaN
away_avg_rush_yards                              NaN
away_avg_third_down_attempts                     NaN
away_avg_third_down_conversions                  NaN
away_avg_time_of_possession                      NaN
away_avg_times_sacked                            NaN
away_avg_first_downs_allowed                     NaN
away_avg_def_fourth_down_attempts             

In [333]:
new_box_avgs_away = pd.DataFrame(columns = desired_columns_away)
new_box_avgs_home = pd.DataFrame(columns = desired_columns_home)
vAvgStats = np.vectorize(findAvgStats)
for i in range(season_and_box.shape[0]):
    new_box_avgs_away = new_box_avgs_away.append(findAvgStats(season_and_box.iloc[i].year, 
                                                              season_and_box.iloc[i].week,
                                                              season_and_box.iloc[i].boxscore,
                                                              season_and_box.iloc[i].away_name,
                                                              atts,
                                                              'away'), 
                                                 ignore_index = True)
for i in range(season_and_box.shape[0]):
    new_box_avgs_home = new_box_avgs_home.append(findAvgStats(season_and_box.iloc[i].year, 
                                                              season_and_box.iloc[i].week,
                                                              season_and_box.iloc[i].boxscore,
                                                              season_and_box.iloc[i].home_name,
                                                              atts,
                                                              'home'), 
                                                 ignore_index = True)
total_box = new_box_avgs_away.merge(new_box_avgs_home, on = ['year', 'week', 'boxscore'], how = 'inner')
total_box

/Users/sethfriman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in long_scalars
/Users/sethfriman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in long_scalars
/Users/sethfriman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in double_scalars
/Users/sethfriman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in double_scalars


,year,week,boxscore,away_avg_first_downs,away_avg_fourth_down_attempts,away_avg_fourth_down_conversions,away_avg_fumbles_lost,away_avg_interceptions_thrown,away_avg_pass_touchdowns,away_avg_pass_yards,...,home_avg_def_fourth_down_converstions,home_avg_fumbles_recovered,home_avg_interceptions_taken,home_avg_pass_touchdowns_allowed,home_avg_pass_yards_allowed,home_avg_rush_touchdowns_allowed,home_avg_rush_yards_allowed,home_avg_def_third_down_attempts,home_avg_def_third_down_converstions,home_avg_sacks
0,2015,1,201509100nwe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,201509130chi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,201509130ram,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,201509130was,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,201509130htx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2015,1,201509130buf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2015,1,201509130jax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2015,1,201509130nyj,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2015,1,201509130sdg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2015,1,201509130crd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [334]:
total_box.iloc[453]

year                                             2016
week                                               14
boxscore                                 201612110mia
away_avg_first_downs                            23.25
away_avg_fourth_down_attempts                0.583333
away_avg_fourth_down_conversions                 0.25
away_avg_fumbles_lost                             0.5
away_avg_interceptions_thrown                 1.08333
away_avg_pass_touchdowns                      1.66667
away_avg_pass_yards                           284.583
away_avg_penalties                            6.58333
away_avg_rush_touchdowns                            1
away_avg_rush_yards                            105.25
away_avg_third_down_attempts                       14
away_avg_third_down_conversions               6.08333
away_avg_time_of_possession                   32.2722
away_avg_times_sacked                         2.91667
away_avg_first_downs_allowed                  17.6667
away_avg_def_fourth_down_att

In [335]:
total_box['away_fourth_down_converstion_pct'] = (total_box['away_avg_fourth_down_conversions'] / 
                                                 total_box['away_avg_fourth_down_attempts'])
total_box['away_third_down_converstion_pct'] = (total_box['away_avg_third_down_conversions'] / 
                                                 total_box['away_avg_third_down_attempts'])
total_box['away_def_fourth_down_converstion_pct'] = (total_box['away_avg_def_fourth_down_conversions'] / 
                                                 total_box['away_avg_def_fourth_down_attempts'])
total_box['away_def_third_down_converstion_pct'] = (total_box['away_avg_def_third_down_conversions'] / 
                                                 total_box['away_avg_def_third_down_attempts'])

total_box['home_fourth_down_converstion_pct'] = (total_box['home_avg_fourth_down_converstions'] / 
                                                 total_box['home_avg_fourth_down_attempts'])
total_box['home_third_down_converstion_pct'] = (total_box['home_avg_third_down_converstions'] / 
                                                 total_box['home_avg_third_down_attempts'])
total_box['home_def_fourth_down_converstion_pct'] = (total_box['home_avg_def_fourth_down_converstions'] / 
                                                 total_box['home_avg_def_fourth_down_attempts'])
total_box['home_def_third_down_converstion_pct'] = (total_box['home_avg_def_third_down_converstions'] / 
                                                 total_box['home_avg_def_third_down_attempts'])
total_box

,year,week,boxscore,away_avg_first_downs,away_avg_fourth_down_attempts,away_avg_fourth_down_conversions,away_avg_fumbles_lost,away_avg_interceptions_thrown,away_avg_pass_touchdowns,away_avg_pass_yards,...,home_avg_def_third_down_converstions,home_avg_sacks,away_fourth_down_converstion_pct,away_third_down_converstion_pct,away_def_fourth_down_converstion_pct,away_def_third_down_converstion_pct,home_fourth_down_converstion_pct,home_third_down_converstion_pct,home_def_fourth_down_converstion_pct,home_def_third_down_converstion_pct
0,2015,1,201509100nwe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,201509130chi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,201509130ram,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,201509130was,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,201509130htx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2015,1,201509130buf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2015,1,201509130jax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2015,1,201509130nyj,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2015,1,201509130sdg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2015,1,201509130crd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [336]:
complete_dataframe = games_and_lines.merge(total_box, on = ['year', 'week', 'boxscore'], how = 'inner')
complete_dataframe

,year,week,boxscore,away_name,away_score,home_name,home_score,losing_name,winning_name,home_wins,...,home_avg_def_third_down_converstions,home_avg_sacks,away_fourth_down_converstion_pct,away_third_down_converstion_pct,away_def_fourth_down_converstion_pct,away_def_third_down_converstion_pct,home_fourth_down_converstion_pct,home_third_down_converstion_pct,home_def_fourth_down_converstion_pct,home_def_third_down_converstion_pct
0,2015,1,201509100nwe,Pittsburgh Steelers,21,New England Patriots,28,Pittsburgh Steelers,New England Patriots,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,201509130chi,Green Bay Packers,31,Chicago Bears,23,Chicago Bears,Green Bay Packers,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,201509130ram,Seattle Seahawks,31,St. Louis Rams,34,Seattle Seahawks,St. Louis Rams,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,201509130was,Miami Dolphins,17,Washington Redskins,10,Washington Redskins,Miami Dolphins,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,201509130htx,Kansas City Chiefs,27,Houston Texans,20,Houston Texans,Kansas City Chiefs,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2015,1,201509130buf,Indianapolis Colts,14,Buffalo Bills,27,Indianapolis Colts,Buffalo Bills,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2015,1,201509130jax,Carolina Panthers,20,Jacksonville Jaguars,9,Jacksonville Jaguars,Carolina Panthers,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2015,1,201509130nyj,Cleveland Browns,10,New York Jets,31,Cleveland Browns,New York Jets,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2015,1,201509130sdg,Detroit Lions,28,San Diego Chargers,33,Detroit Lions,San Diego Chargers,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2015,1,201509130crd,New Orleans Saints,19,Arizona Cardinals,31,New Orleans Saints,Arizona Cardinals,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [337]:
# spelling mistakes
complete_dataframe = complete_dataframe.rename(
    columns = {'home_avg_fourth_down_converstions' : 'home_avg_fourth_down_conversions', 
               'home_avg_third_down_converstions' : 'home_avg_third_down_conversions', 
               'home_avg_def_fourth_down_converstions' : 'home_avg_def_fourth_down_conversions',
               'home_avg_def_third_down_converstions' : 'home_avg_def_third_down_conversions',
               'away_fourth_down_converstion_pct' : 'away_fourth_down_conversion_pct', 
               'away_third_down_converstion_pct' : 'away_third_down_conversion_pct',
               'away_def_fourth_down_converstion_pct' : 'away_def_fourth_down_conversion_pct',
               'away_def_third_down_converstion_pct' : 'away_def_third_down_conversion_pct',
               'home_fourth_down_converstion_pct' : 'home_fourth_down_conversion_pct', 
               'home_third_down_converstion_pct' : 'home_third_down_conversion_pct',
               'home_def_fourth_down_converstion_pct' : 'home_def_fourth_down_conversion_pct',
               'home_def_third_down_converstion_pct' : 'home_def_third_down_conversion_pct'})
complete_dataframe.columns

Index(['year', 'week', 'boxscore', 'away_name', 'away_score', 'home_name',
       'home_score', 'losing_name', 'winning_name', 'home_wins', 'home_losses',
       'away_wins', 'away_losses', 'home_streak_before_game',
       'away_streak_before_game', 'division_matchup', 'home_last_season_wins',
       'away_last_season_wins', 'home_win', 'favorite_name', 'spread_favorite',
       'over_under_line', 'home_favorite', 'away_avg_first_downs',
       'away_avg_fourth_down_attempts', 'away_avg_fourth_down_conversions',
       'away_avg_fumbles_lost', 'away_avg_interceptions_thrown',
       'away_avg_pass_touchdowns', 'away_avg_pass_yards', 'away_avg_penalties',
       'away_avg_rush_touchdowns', 'away_avg_rush_yards',
       'away_avg_third_down_attempts', 'away_avg_third_down_conversions',
       'away_avg_time_of_possession', 'away_avg_times_sacked',
       'away_avg_first_downs_allowed', 'away_avg_def_fourth_down_attempts',
       'away_avg_def_fourth_down_conversions', 'away_avg_fumbles_

In [338]:
# At this point I should probably split this into 2 dataframes
# First one will be a difference of home stats - away stats to answer the question 'will home win?'
# - need to find all relevant stats and change spread_favorite to be points added to the home team based 
#   on the home_favorite variable, then drop some categories
# - all new columns will be home stat - away stat and need to be consistent

# Second dataframe will keep all stats as they are to answer 'how many points do we predict they will score'
# - need to run 2 models, one for home team and one for away based on very similar data

In [339]:
# Changing spread_favorite to home_line for consistency and a number that can be used on its own
complete_dataframe = complete_dataframe.rename(
    columns = {'spread_favorite' : 'home_line'})

In [340]:
def changeLine(line, fav) :
    if fav == 0 :
        return line * -1
    else:
        return line

In [341]:
vChangeLine = np.vectorize(changeLine)
complete_dataframe['home_line'] = vChangeLine(complete_dataframe.home_line.values, 
                                              complete_dataframe.home_favorite.values)
complete_dataframe

,year,week,boxscore,away_name,away_score,home_name,home_score,losing_name,winning_name,home_wins,...,home_avg_def_third_down_conversions,home_avg_sacks,away_fourth_down_conversion_pct,away_third_down_conversion_pct,away_def_fourth_down_conversion_pct,away_def_third_down_conversion_pct,home_fourth_down_conversion_pct,home_third_down_conversion_pct,home_def_fourth_down_conversion_pct,home_def_third_down_conversion_pct
0,2015,1,201509100nwe,Pittsburgh Steelers,21,New England Patriots,28,Pittsburgh Steelers,New England Patriots,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,201509130chi,Green Bay Packers,31,Chicago Bears,23,Chicago Bears,Green Bay Packers,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,201509130ram,Seattle Seahawks,31,St. Louis Rams,34,Seattle Seahawks,St. Louis Rams,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,201509130was,Miami Dolphins,17,Washington Redskins,10,Washington Redskins,Miami Dolphins,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,201509130htx,Kansas City Chiefs,27,Houston Texans,20,Houston Texans,Kansas City Chiefs,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2015,1,201509130buf,Indianapolis Colts,14,Buffalo Bills,27,Indianapolis Colts,Buffalo Bills,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2015,1,201509130jax,Carolina Panthers,20,Jacksonville Jaguars,9,Jacksonville Jaguars,Carolina Panthers,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2015,1,201509130nyj,Cleveland Browns,10,New York Jets,31,Cleveland Browns,New York Jets,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2015,1,201509130sdg,Detroit Lions,28,San Diego Chargers,33,Detroit Lions,San Diego Chargers,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2015,1,201509130crd,New Orleans Saints,19,Arizona Cardinals,31,New Orleans Saints,Arizona Cardinals,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [342]:
complete_dataframe.columns

Index(['year', 'week', 'boxscore', 'away_name', 'away_score', 'home_name',
       'home_score', 'losing_name', 'winning_name', 'home_wins', 'home_losses',
       'away_wins', 'away_losses', 'home_streak_before_game',
       'away_streak_before_game', 'division_matchup', 'home_last_season_wins',
       'away_last_season_wins', 'home_win', 'favorite_name', 'home_line',
       'over_under_line', 'home_favorite', 'away_avg_first_downs',
       'away_avg_fourth_down_attempts', 'away_avg_fourth_down_conversions',
       'away_avg_fumbles_lost', 'away_avg_interceptions_thrown',
       'away_avg_pass_touchdowns', 'away_avg_pass_yards', 'away_avg_penalties',
       'away_avg_rush_touchdowns', 'away_avg_rush_yards',
       'away_avg_third_down_attempts', 'away_avg_third_down_conversions',
       'away_avg_time_of_possession', 'away_avg_times_sacked',
       'away_avg_first_downs_allowed', 'away_avg_def_fourth_down_attempts',
       'away_avg_def_fourth_down_conversions', 'away_avg_fumbles_recove

In [343]:
# dropping 2015 week 1 because we don't have stats for it
complete_dataframe = complete_dataframe[16:]
complete_dataframe = complete_dataframe

In [345]:
# All new columns are home - away
# outcome will be binary 'home_win'

logistic_dataframe = complete_dataframe[['year', 'week', 'boxscore', 'home_name', 'away_name', 'home_line']]
# win percentage diff will be weighted by week
#i.e. a win diff of 1 or -1 will mean a lot more in week 16 than it would in week 2
# Constant 4 is in there to not exaggerate difference by too much
logistic_dataframe['weighted_win_pct_diff'] = (((complete_dataframe['home_wins'] / 
                                      (complete_dataframe['home_wins'] + complete_dataframe['home_losses'])) - 
                                      (complete_dataframe['away_wins'] / 
                                      (complete_dataframe['away_wins'] + complete_dataframe['away_losses']))) * 
                                      (complete_dataframe['week'] / 4))
logistic_dataframe['streak_diff'] = (complete_dataframe['home_streak_before_game'] - 
                                     complete_dataframe['away_streak_before_game'])
logistic_dataframe['last_season_win_diff'] = (complete_dataframe['home_last_season_wins'] - 
                                              complete_dataframe['away_last_season_wins'])
#This and other columns will compute (home off value - home def value) - (away off value - away def value)
# comes out to (home net) - (away net)
logistic_dataframe['first_down_diff'] = ((complete_dataframe['home_avg_first_downs'] - 
                                              complete_dataframe['home_avg_first_downs_allowed']) - 
                                             (complete_dataframe['away_avg_first_downs'] - 
                                              complete_dataframe['away_avg_first_downs_allowed']))
logistic_dataframe['fourth_down_conv_diff'] = ((complete_dataframe['home_fourth_down_conversion_pct'] - 
                                              complete_dataframe['home_def_fourth_down_conversion_pct']) - 
                                             (complete_dataframe['away_fourth_down_conversion_pct'] - 
                                              complete_dataframe['away_def_fourth_down_conversion_pct']))
logistic_dataframe['third_down_conv_diff'] = ((complete_dataframe['home_third_down_conversion_pct'] - 
                                              complete_dataframe['home_def_third_down_conversion_pct']) - 
                                             (complete_dataframe['away_third_down_conversion_pct'] - 
                                              complete_dataframe['away_def_third_down_conversion_pct']))
logistic_dataframe['fumble_diff'] = ((complete_dataframe['home_avg_fumbles_recovered'] - 
                                              complete_dataframe['home_avg_fumbles_lost']) - 
                                             (complete_dataframe['away_avg_fumbles_recovered'] - 
                                              complete_dataframe['away_avg_fumbles_lost']))
logistic_dataframe['interception_diff'] = ((complete_dataframe['home_avg_interceptions_taken'] - 
                                      complete_dataframe['home_avg_interceptions_thrown']) - 
                                      (complete_dataframe['away_avg_interceptions_taken'] - 
                                      complete_dataframe['away_avg_interceptions_thrown']))
logistic_dataframe['pass_td_diff'] = ((complete_dataframe['home_avg_pass_touchdowns'] - 
                                      complete_dataframe['home_avg_pass_touchdowns_allowed']) - 
                                      (complete_dataframe['away_avg_pass_touchdowns'] - 
                                      complete_dataframe['away_avg_pass_touchdowns_allowed']))
logistic_dataframe['rush_td_diff'] = ((complete_dataframe['home_avg_rush_touchdowns'] - 
                                      complete_dataframe['home_avg_rush_touchdowns_allowed']) - 
                                      (complete_dataframe['away_avg_rush_touchdowns'] - 
                                      complete_dataframe['away_avg_rush_touchdowns_allowed']))
logistic_dataframe['pass_yards_diff'] = ((complete_dataframe['home_avg_pass_yards'] - 
                                      complete_dataframe['home_avg_pass_yards_allowed']) - 
                                      (complete_dataframe['away_avg_pass_yards'] - 
                                      complete_dataframe['away_avg_pass_yards_allowed']))
logistic_dataframe['rush_yards_diff'] = ((complete_dataframe['home_avg_rush_yards'] - 
                                      complete_dataframe['home_avg_rush_yards_allowed']) - 
                                      (complete_dataframe['away_avg_rush_yards'] - 
                                      complete_dataframe['away_avg_rush_yards_allowed']))
logistic_dataframe['penalties_diff'] = (complete_dataframe['home_avg_penalties'] - 
                                        complete_dataframe['away_avg_penalties'])
logistic_dataframe['time_of_poss_diff'] = (complete_dataframe['home_avg_time_of_possession'] - 
                                        complete_dataframe['away_avg_time_of_possession'])
logistic_dataframe['sack_diff'] = ((complete_dataframe['home_avg_sacks'] - 
                                      complete_dataframe['home_avg_times_sacked']) - 
                                      (complete_dataframe['away_avg_sacks'] - 
                                      complete_dataframe['away_avg_times_sacked']))
logistic_dataframe['home_win'] = complete_dataframe['home_win']
logistic_dataframe

/Users/sethfriman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/Users/sethfriman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/sethfriman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

,year,week,boxscore,home_name,away_name,home_line,weighted_win_pct_diff,streak_diff,last_season_win_diff,first_down_diff,...,fumble_diff,interception_diff,pass_td_diff,rush_td_diff,pass_yards_diff,rush_yards_diff,penalties_diff,time_of_poss_diff,sack_diff,home_win
16,2015,2,201509170kan,Kansas City Chiefs,Denver Broncos,-3.0,0,0,0,-10.0,...,1.0,0.0,1.0,0.0,-149.0,3.0,-6.0,-1.966667,5.0,0
17,2015,2,201509200chi,Chicago Bears,Arizona Cardinals,2.0,-0.5,-2,0,-3.0,...,1.0,-2.0,-4.0,-1.0,84.0,-10.0,1.0,5.266667,-4.0,0
18,2015,2,201509200car,Carolina Panthers,Houston Texans,-3.0,0.5,2,0,-6.0,...,2.0,2.0,1.0,0.0,-99.0,8.0,-3.0,9.416667,6.0,1
19,2015,2,201509200buf,Buffalo Bills,New England Patriots,-1.0,0,0,0,-11.0,...,1.0,1.0,-4.0,3.0,15.0,137.0,4.0,3.016667,1.0,0
20,2015,2,201509200pit,Pittsburgh Steelers,San Francisco 49ers,-6.0,-0.5,-2,0,-11.0,...,1.0,-2.0,-3.0,-1.0,129.0,-105.0,0.0,-0.950000,-5.0,1
21,2015,2,201509200nyg,New York Giants,Atlanta Falcons,-2.0,-0.5,-2,0,-7.0,...,1.0,2.0,-4.0,3.0,-129.0,-23.0,-2.0,-12.616667,0.0,0
22,2015,2,201509200nor,New Orleans Saints,Tampa Bay Buccaneers,-9.0,0,0,0,-3.0,...,0.0,1.0,0.0,0.0,47.0,-34.0,-5.0,4.900000,0.0,0
23,2015,2,201509200cle,Cleveland Browns,Tennessee Titans,1.0,-0.5,-2,0,-7.0,...,-3.0,-2.0,-3.0,-3.0,53.0,-82.0,4.0,0.166667,-5.0,1
24,2015,2,201509200was,Washington Redskins,St. Louis Rams,3.0,-0.5,-2,0,3.0,...,4.0,-3.0,0.0,-2.0,-76.0,135.0,7.0,9.366667,-2.0,1
25,2015,2,201509200min,Minnesota Vikings,Detroit Lions,-2.5,0,0,0,4.0,...,0.0,-1.0,0.0,-1.0,223.0,-132.0,1.0,5.166667,-5.0,1


In [346]:
only_2019 = logistic_dataframe[logistic_dataframe['year'] < 2020]

In [351]:
only_2019 = only_2019.fillna(0)

In [356]:
only_2019

,year,week,boxscore,home_name,away_name,home_line,weighted_win_pct_diff,streak_diff,last_season_win_diff,first_down_diff,...,fumble_diff,interception_diff,pass_td_diff,rush_td_diff,pass_yards_diff,rush_yards_diff,penalties_diff,time_of_poss_diff,sack_diff,home_win
16,2015,2,201509170kan,Kansas City Chiefs,Denver Broncos,-3.0,0.000000,0,0,-10.000000,...,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,-149.000000,3.000000,-6.000000,-1.966667,5.000000,0
17,2015,2,201509200chi,Chicago Bears,Arizona Cardinals,2.0,-0.500000,-2,0,-3.000000,...,1.000000e+00,-2.000000e+00,-4.000000e+00,-1.000000e+00,84.000000,-10.000000,1.000000,5.266667,-4.000000,0
18,2015,2,201509200car,Carolina Panthers,Houston Texans,-3.0,0.500000,2,0,-6.000000,...,2.000000e+00,2.000000e+00,1.000000e+00,0.000000e+00,-99.000000,8.000000,-3.000000,9.416667,6.000000,1
19,2015,2,201509200buf,Buffalo Bills,New England Patriots,-1.0,0.000000,0,0,-11.000000,...,1.000000e+00,1.000000e+00,-4.000000e+00,3.000000e+00,15.000000,137.000000,4.000000,3.016667,1.000000,0
20,2015,2,201509200pit,Pittsburgh Steelers,San Francisco 49ers,-6.0,-0.500000,-2,0,-11.000000,...,1.000000e+00,-2.000000e+00,-3.000000e+00,-1.000000e+00,129.000000,-105.000000,0.000000,-0.950000,-5.000000,1
21,2015,2,201509200nyg,New York Giants,Atlanta Falcons,-2.0,-0.500000,-2,0,-7.000000,...,1.000000e+00,2.000000e+00,-4.000000e+00,3.000000e+00,-129.000000,-23.000000,-2.000000,-12.616667,0.000000,0
22,2015,2,201509200nor,New Orleans Saints,Tampa Bay Buccaneers,-9.0,0.000000,0,0,-3.000000,...,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,47.000000,-34.000000,-5.000000,4.900000,0.000000,0
23,2015,2,201509200cle,Cleveland Browns,Tennessee Titans,1.0,-0.500000,-2,0,-7.000000,...,-3.000000e+00,-2.000000e+00,-3.000000e+00,-3.000000e+00,53.000000,-82.000000,4.000000,0.166667,-5.000000,1
24,2015,2,201509200was,Washington Redskins,St. Louis Rams,3.0,-0.500000,-2,0,3.000000,...,4.000000e+00,-3.000000e+00,0.000000e+00,-2.000000e+00,-76.000000,135.000000,7.000000,9.366667,-2.000000,1
25,2015,2,201509200min,Minnesota Vikings,Detroit Lions,-2.5,0.000000,0,0,4.000000,...,0.000000e+00,-1.000000e+00,0.000000e+00,-1.000000e+00,223.000000,-132.000000,1.000000,5.166667,-5.000000,1


In [360]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [361]:
logistic_dataframe.columns

Index(['year', 'week', 'boxscore', 'home_name', 'away_name', 'home_line',
       'weighted_win_pct_diff', 'streak_diff', 'last_season_win_diff',
       'first_down_diff', 'fourth_down_conv_diff', 'third_down_conv_diff',
       'fumble_diff', 'interception_diff', 'pass_td_diff', 'rush_td_diff',
       'pass_yards_diff', 'rush_yards_diff', 'penalties_diff',
       'time_of_poss_diff', 'sack_diff', 'home_win'],
      dtype='object')

In [414]:
#Creating the model
feature_cols = ['home_line',
       'weighted_win_pct_diff', 'streak_diff', 'last_season_win_diff',
       'first_down_diff', 'fourth_down_conv_diff', 'third_down_conv_diff',
       'fumble_diff', 'interception_diff', 'pass_td_diff', 'rush_td_diff',
       'pass_yards_diff', 'rush_yards_diff', 'penalties_diff',
       'time_of_poss_diff', 'sack_diff']
test_data = only_2019[feature_cols]
X_train, X_test, y_train, y_test = train_test_split(test_data, only_2019.home_win, test_size=0.2)

In [415]:
logreg = LogisticRegression()


logreg.fit(X_train, y_train)

/Users/sethfriman/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [416]:
logreg.score(X_test, y_test)

0.6390041493775933

In [417]:
coef_dict = {}
for coef, feat in zip(logreg.coef_[0,:],feature_cols):
    coef_dict[feat] = coef
    
coef_dict

{'home_line': -0.10890838984407625,
 'weighted_win_pct_diff': 0.03047125996794295,
 'streak_diff': 0.033396781809631015,
 'last_season_win_diff': 0.012162665400195614,
 'first_down_diff': -0.001406581161671447,
 'fourth_down_conv_diff': -0.12146917502478727,
 'third_down_conv_diff': 0.17791070425393454,
 'fumble_diff': 0.07325914850081966,
 'interception_diff': 0.08632913836576941,
 'pass_td_diff': -0.03370074973404919,
 'rush_td_diff': -0.14064699797066105,
 'pass_yards_diff': -0.0005394313775701692,
 'rush_yards_diff': 0.0006999342611789548,
 'penalties_diff': -0.004215520069746005,
 'time_of_poss_diff': 0.01072580660493632,
 'sack_diff': 0.04617206227475185}

In [418]:
import statsmodels.api as sm
sm_model = sm.Logit(y_train, sm.add_constant(X_train)).fit(disp=0)
print(sm_model.pvalues)
sm_model.summary()

const                    1.563494e-01
home_line                3.871534e-09
weighted_win_pct_diff    8.490272e-01
streak_diff              1.582089e-01
last_season_win_diff     5.449737e-01
first_down_diff          9.579135e-01
fourth_down_conv_diff    4.801782e-01
third_down_conv_diff     7.107007e-01
fumble_diff              4.494139e-01
interception_diff        3.421183e-01
pass_td_diff             6.992261e-01
rush_td_diff             2.579822e-01
pass_yards_diff          7.885750e-01
rush_yards_diff          7.617614e-01
penalties_diff           9.266572e-01
time_of_poss_diff        8.288998e-01
sack_diff                3.216431e-01
dtype: float64


/Users/sethfriman/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               home_win   No. Observations:                  960
Model:                          Logit   Df Residuals:                      943
Method:                           MLE   Df Model:                           16
Date:                Sat, 03 Oct 2020   Pseudo R-squ.:                 0.09987
Time:                        13:30:10   Log-Likelihood:                -591.97
converged:                       True   LL-Null:                       -657.65
Covariance Type:            nonrobust   LLR p-value:                 3.497e-20
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     0.1092      0.077      1.417      0.156      -0.042       0.260
home_line                -0.1089      0.018     -5.890      0.000      -0.145      -0.073
weighted_win_pct_diff     0.0303      0.159      0.190      0.849      -0.282       0.342
streak_diff               0.0335      0.024      1.411      0.158      -0.013       0.080
last_season_win_diff      0.0121      0.020      0.605      0.545      -0.027       0.051
first_down_diff          -0.0016      0.031     -0.053      0.958      -0.063       0.059
fourth_down_conv_diff    -0.1210      0.171     -0.706      0.480      -0.457       0.215
third_down_conv_diff      0.2868      0.773      0.371      0.711      -1.229       1.802
fumble_diff               0.0762      0.101      0.756      0.449      -0.121       0.274
interception_diff         0.0885      0.093      0.950      0.342      -0.094       0.271
pass_td_diff             -0.0399      0.103     -0.386      0.699      -0.242       0.163
rush_td_diff             -0.1477      0.131     -1.131      0.258      -0.404       0.108
pass_yards_diff          -0.0005      0.002     -0.268      0.789      -0.004       0.003
rush_yards_diff           0.0007      0.002      0.303      0.762      -0.004       0.005
penalties_diff           -0.0039      0.042     -0.092      0.927      -0.087       0.079
time_of_poss_diff         0.0086      0.040      0.216      0.829      -0.069       0.086
sack_diff                 0.0459      0.046      0.991      0.322      -0.045       0.137
=========================================================================================
"""

In [419]:
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix

In [420]:
pred = logreg.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
(tn, fp, fn, tp)

(56, 51, 36, 98)

In [421]:
# Beginning the analysis for predicting home score

In [ ]:
cols = ['year', 'week', 'boxscore', 'away_name', 'away_score', 'home_name',
       'home_score', 'losing_name', 'winning_name', 'home_wins', 'home_losses',
       'away_wins', 'away_losses', 'home_streak_before_game',
       'away_streak_before_game', 'division_matchup', 'home_last_season_wins',
       'away_last_season_wins', 'home_win', 'favorite_name', 'home_line',
       'over_under_line', 'home_favorite', 'away_avg_first_downs',
       'away_avg_fourth_down_attempts', 'away_avg_fourth_down_conversions',
       'away_avg_fumbles_lost', 'away_avg_interceptions_thrown',
       'away_avg_pass_touchdowns', 'away_avg_pass_yards', 'away_avg_penalties',
       'away_avg_rush_touchdowns', 'away_avg_rush_yards',
       'away_avg_third_down_attempts', 'away_avg_third_down_conversions',
       'away_avg_time_of_possession', 'away_avg_times_sacked',
       'away_avg_first_downs_allowed', 'away_avg_def_fourth_down_attempts',
       'away_avg_def_fourth_down_conversions', 'away_avg_fumbles_recovered',
       'away_avg_interceptions_taken', 'away_avg_pass_touchdowns_allowed',
       'away_avg_pass_yards_allowed', 'away_avg_rush_touchdowns_allowed',
       'away_avg_rush_yards_allowed', 'away_avg_def_third_down_attempts',
       'away_avg_def_third_down_conversions', 'away_avg_sacks',
       'home_avg_first_downs', 'home_avg_fourth_down_attempts',
       'home_avg_fourth_down_conversions', 'home_avg_fumbles_lost',
       'home_avg_interceptions_thrown', 'home_avg_pass_touchdowns',
       'home_avg_pass_yards', 'home_avg_penalties', 'home_avg_rush_touchdowns',
       'home_avg_rush_yards', 'home_avg_third_down_attempts',
       'home_avg_third_down_conversions', 'home_avg_time_of_possession',
       'home_avg_times_sacked', 'home_avg_first_downs_allowed',
       'home_avg_def_fourth_down_attempts',
       'home_avg_def_fourth_down_conversions', 'home_avg_fumbles_recovered',
       'home_avg_interceptions_taken', 'home_avg_pass_touchdowns_allowed',
       'home_avg_pass_yards_allowed', 'home_avg_rush_touchdowns_allowed',
       'home_avg_rush_yards_allowed', 'home_avg_def_third_down_attempts',
       'home_avg_def_third_down_conversions', 'home_avg_sacks',
       'away_fourth_down_conversion_pct', 'away_third_down_conversion_pct',
       'away_def_fourth_down_conversion_pct',
       'away_def_third_down_conversion_pct', 'home_fourth_down_conversion_pct',
       'home_third_down_conversion_pct', 'home_def_fourth_down_conversion_pct',
       'home_def_third_down_conversion_pct']

linear_df_home = complete_dataframe['year', 'week', 'boxscore', 'home_name', 'division_matchup', 
                                    'over_under_line', 'home_favorite', ]